# Econometrics 1 

## TD 3 - 02/10/2024

TA: Pedro Vergara Merino ([pedro.vergaramerino@ensae.fr](mailto:pedro.vergaramerino@ensae.fr), office 4081) 

In [2]:
rm(list=ls()) #Clean workspace

if(!require(car)) install.packages("car")
if(!require(estimatr)) install.packages("estimatr")

set.seed(100) #Select a random seed in order to replicate results


Le chargement a nécessité le package : car

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“aucun package nommé ‘car’ n'est trouvé”
installation des dépendances ‘nloptr’, ‘pbkrtest’, ‘lme4’


Warning message in install.packages("car"):
“l'installation du package ‘nloptr’ a eu un statut de sortie non nul”
Warning message in install.packages("car"):
“l'installation du package ‘lme4’ a eu un statut de sortie non nul”
Warning message in install.packages("car"):
“l'installation du package ‘pbkrtest’ a eu un statut de sortie non nul”
Warning message in install.packages("car"):
“l'installation du package ‘car’ a eu un statut de sortie non nul”
Mise à jour de la liste HTML des packages dans '.Library'

Making 'packages.html' ...
 terminé

Le chargement a nécessité le package : estimatr



1. Simulate data from the model, then estimate the linear regression of `Y` on `D` and a constant for samples of different sizes: `n = 20`, `n = 100`, `n = 10^3`, `n = 10^5`. What do you observe for $(\hat{\beta}_{1,n}, \hat{\beta}_{2,n})$? Which property of the OLS estimator is being illustrated here?


In [3]:
#Generate the data and perform regressions for every sample size
for (n in c(20,100,10^3,10^5))
{
  cat("Sample size : ", n, "\n") 
  #1. We generate one random sample
  D=rnorm(n) #Generate D from a standard normal distribution
  U=runif(n, min=-1, max=1) #Generate U from a continuous uniform distribution
  Y=1+D+(1+D)*U 
  
  #2. We estimate the model using OLS
  model=lm(Y~D)
  print(coef(summary(model)))
  cat("\n")
}

Sample size :  20 
             Estimate Std. Error  t value     Pr(>|t|)
(Intercept) 0.9372599  0.1654121 5.666210 2.248477e-05
D           0.7174188  0.2334259 3.073432 6.548789e-03

Sample size :  100 
            Estimate Std. Error  t value     Pr(>|t|)
(Intercept) 1.132583 0.07729969 14.65185 1.973875e-26
D           1.223718 0.07434799 16.45932 5.777361e-30

Sample size :  1000 
            Estimate Std. Error  t value      Pr(>|t|)
(Intercept) 1.015859 0.02595191 39.14391 7.922591e-204
D           1.015729 0.02494657 40.71617 2.491594e-214

Sample size :  1e+05 
             Estimate  Std. Error  t value Pr(>|t|)
(Intercept) 0.9976015 0.002570755 388.0577        0
D           1.0000378 0.002574305 388.4690        0



2. Same question when we draw $U$ from a Cauchy distribution, that is, with a density with respect to Lebesgue’s measure equal to 

$$
f: \mathbb{R} \rightarrow \mathbb{R}^+, \quad f(x) := \frac{1}{\pi(1+x^2)}.
$$


3. Henceforth, we take $ U \sim \text{Uniform} $ again. Draw $ S = 10^3 $ samples of size $ n = 20 $ and store the obtained OLS estimates of the slope $ \hat{\beta}_{2,n}^{(1)}, \ldots, \hat{\beta}_{2,n}^{(S)} $, then plot the empirical density of those estimates. Do the same for samples of size $ n = 1,000 $, and compare the results. What do you remark? Which property of the OLS estimator do we illustrate here?

